In [14]:
import requests
import time 
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
import re

### 각 지점 Name, ID 얻기

In [15]:
driver = webdriver.Chrome()
driver.get('https://www.khoa.go.kr/swtc/mobile.do')
node = driver.find_elements(By.XPATH, '//*[@id="gisSelect"]/option')
obsList = []
for i in node :
    obsList.append ( [i.text, i.get_attribute('value') ])
driver.close()

In [16]:
print('sample [54] : ',obsList[54][0], obsList[54][1])
print('len : ',len(obsList))

sample [54] :  말도 SO_0547
len :  166


### 목표값 json 데이터 형태 확인

In [17]:
url = 'https://www.khoa.go.kr/swtc/getWeeklyData.do?stDate=20250322&obsPostId=DT_0059'
response = requests.get(url)
data = response.json()
data

{'getWeeklyData': {'weeklyData': [{'searchDate': '2025-03-22',
    'obsPostName': '백령도',
    'obsLon': 124.736081,
    'obsLat': 37.95565,
    'lvl1': '04:16/low/저/93',
    'lvl2': '11:10/high/고/310',
    'lvl3': '17:41/low/저/176',
    'lvl4': '22:18/high/고/233',
    'flgView': 'Y',
    'dateSun': '2025/03/22',
    'dateMoon': '2025/02/23',
    'moonState': '소조기',
    'moolNormal': '선조금',
    'mool7': '조금',
    'mool8': '조금',
    'temp': '/'},
   {'searchDate': '2025-03-23',
    'obsPostName': '백령도',
    'obsLon': 124.736081,
    'obsLat': 37.95565,
    'lvl1': '04:59/low/저/114',
    'lvl2': '12:32/high/고/300',
    'lvl3': '19:34/low/저/190',
    'lvl4': '23:47/high/고/211',
    'flgView': 'Y',
    'dateSun': '2025/03/23',
    'dateMoon': '2025/02/24',
    'moonState': '소조기',
    'moolNormal': '앉은조금',
    'mool7': '무시',
    'mool8': '한물',
    'temp': '/'},
   {'searchDate': '2025-03-24',
    'obsPostName': '백령도',
    'obsLon': 124.736081,
    'obsLat': 37.95565,
    'lvl1': '06:16/low/저/

In [19]:
for i in data :
    print(i)

getWeeklyData
weeklyDataDTO
weeklyData


In [20]:
data['weeklyData'][5]

{'searchDate': '2025-03-27',
 'obsPostName': '백령도',
 'obsLon': 124.736081,
 'obsLat': 37.95565,
 'lvl1': '04:30/high/고/280',
 'lvl2': '10:55/low/저/70',
 'lvl3': '17:10/high/고/365',
 'lvl4': '23:47/low/저/78',
 'flgView': 'Y',
 'dateSun': '2025/03/27',
 'dateMoon': '2025/02/28',
 'moolNormal': '무릎사리',
 'mool7': '네매',
 'mool8': '다섯물',
 'temp': '/'}

In [73]:
datetime.strptime('1:09:44 PM', '%I:%M:%S %p').strftime("%H:%M")
datetime.strptime('20250325', '%Y%m%d').strftime("%Y-%m-%d")

'13:09'

In [9]:
def getData(obs, code):

    obsName = obs[0]
    obsId = obs[1]
    
    start_date = datetime(2025, 3, 1)
    end_date = datetime(2026, 12, 31)
    
    current_date = start_date
    while current_date <= end_date:
        try :
            dateString = current_date.strftime("%Y%m%d")
            ## 조석 json 데이터 url
            url_tide = f'https://www.khoa.go.kr/swtc/getWeeklyData.do?stDate={dateString}&obsPostId={obsId}'
            response = requests.get(url_tide)
            full_json = response.json()
            result = []
            for data in full_json['weeklyData'] :
                today = data['dateSun']
                today_moon = data['dateMoon']
                mool8 = data['mool8']
                obsPostName = data['obsPostName']
                obsLon = data['obsLon']
                obsLat = data['obsLat']
                lv1 = data['lvl1']
                lv2 = data['lvl2']
                lv3 = data['lvl3']
                lv4 = data['lvl4']
    
                today_dash = datetime.strptime('20250325', '%Y%m%d').strftime("%Y-%m-%d")
                ## 조석지점 위경도의 일출/일몰 데이터 url
                url_sun = f"https://api.sunrise-sunset.org/json?lat={obsLat}&lng={obsLon}&date={today_dash}&tzid=Asia/Seoul"
                sunData = requests.get(url_sun).json()
                sunrise = sunData['results']['sunrise']
                sunset =  sunData['results']['sunset']
                sunrise = datetime.strptime(sunrise, '%I:%M:%S %p').strftime("%H:%M")
                sunset = datetime.strptime(sunset, '%I:%M:%S %p').strftime("%H:%M")
                result.append(f'{obsName},{obsId},{today},{today_moon},{mool8},{obsLon},{obsLat},{lv1},{lv2},{lv3},{lv4},{sunrise},{sunset}')
                
            current_date += timedelta(days=7)

            lock.acquire()
            results.append(result)
            print(f'{obsName}, {today}, {lv1}, {sunrise}, {code}/{len(obsList)}')
            lock.release()
    
        except TypeError : 
            print('***',result)   
        except requests.exceptions.Timeout:
            print(code,'timeout!! ret!')
            #break
            continue


In [ ]:
import threading

for n in range(0, 160, 10) :
    lock = threading.Lock()
    threads = []
    results = []
    for code in range(n, n+10 ): 
        t = threading.Thread(target=getData, args=(obsList[code], code))
        t.start()
        threads.append(t)
        time.sleep(3)
        print('attach',code)

    print('wait...',n)    
    for t in threads:
        t.join()
    
    file = open('.\\TIDE_INFO.txt', 'a')
    for rst in results :
        for i in rst :
            try :
                file.write( f"{i}\n")
            except UnicodeEncodeError : 
                println(i, end=", ************")
    file.close()